# Matched Study - Sensitivity Analysis
This analysis changes the timing criteria to consider the administration of Levophed as opposed to absolute time spent in the ICU

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import math as mt
import matplotlib.pyplot as plt
import psycopg2
from scipy.stats import ks_2samp
from scipy.stats import linregress
from scipy.stats import ttest_ind
from scipy.stats import chisquare
from collections import defaultdict
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
# create a database connection
sqluser = 'KSeverson'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to a local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path TO ' + schema_name)

In [ ]:
#change default figure size
plt.rcParams["figure.figsize"] = (10.0, 5.0)

# Query
Initial MIMIC query focusing on inclusion criteria.

In [ ]:
query = \
"""
SELECT a.hadm_id, a.subject_id, i.icustay_id, ang.angus, i.los, l.valuenum, l.valueuom, a.hospital_expire_flag
	, EXTRACT (EPOCH FROM (i.intime - p.dob)/60.0/60.0/24.0/365.242) AS age_on_admit -- patient age in years
	, EXTRACT (EPOCH FROM (l.charttime - i.intime)/60.0/60.0) AS time_since_admit -- time in hours
    , i.first_careunit, i.last_careunit
    , EXTRACT (EPOCH FROM (p.dod - i.intime)/60.0/60.0) AS time_of_death -- time in hours
FROM admissions a
INNER JOIN patients p
ON a.subject_id = p.subject_id
INNER JOIN labevents l
ON a.hadm_id = l.hadm_id
LEFT JOIN icustays i
ON a.hadm_id = i.hadm_id AND l.charttime BETWEEN (i.intime - '06:00:00'::interval hour) AND i.outtime -- add a buffer time region of 24 hours prior to icu admit 
LEFT JOIN angus_sepsis ang
ON i.hadm_id = ang.hadm_id
WHERE i.first_careunit IN ('MICU', 'SICU')
AND i.last_careunit IN ('MICU', 'SICU')
AND ang.angus = 1
AND l.itemid = 50813
ORDER BY hadm_id, time_since_admit;
"""

data = pd.read_sql_query(query,con)

In [ ]:
# query the metavision table for the medication information
query = \
"""
SELECT icu.icustay_id, ie.itemid, d.label, ie.starttime, ie.rate, ie.rateuom
     , EXTRACT (EPOCH FROM (ie.starttime - icu.intime)/60.0/60.0) AS time_since_admit -- time in hours
FROM icustays icu
LEFT JOIN inputevents_mv ie
ON icu.icustay_id = ie.icustay_id
LEFT JOIN d_items d
ON ie.itemid = d.itemid
WHERE ie.itemid IN (
    221906, 30047, 30120  -- Norepinephrin / Levophed
    ) 
AND ie.rate = ie.rate
ORDER BY icu.icustay_id, ie.starttime
;"""

mvLevo = pd.read_sql_query(query,con)

In [ ]:
# this is the carevue version of the query
query = \
"""
SELECT c.icustay_id, ie.itemid, d.label, ie.charttime, ie.rate, ie.rateuom
     , EXTRACT (EPOCH FROM (ie.charttime - icu.intime)/60.0/60.0) AS time_since_admit -- time in hours
FROM matched_cohort_SICU c 
LEFT JOIN icustays icu
ON c.icustay_id = icu.icustay_id
LEFT JOIN inputevents_cv ie
ON c.icustay_id = ie.icustay_id
LEFT JOIN d_items d
ON ie.itemid = d.itemid
WHERE ie.itemid IN (
    221906, 30047, 30120  -- Norepinephrin / Levophed
    ) 
AND ie.rate = ie.rate
ORDER BY c.icustay_id, ie.charttime
;"""

cvLevo = pd.read_sql_query(query,con)

# Data Processing
This section applies the rest of the inclusion criteria.

In [ ]:
i_id = data.icustay_id #put all of the icu_stay IDs into a variable
#find the unique IDs, their place in the array, and the number of entries for each ID
iID_list, iID_ind, iID_count = np.unique(i_id, return_index = True, return_counts = True) 

In [ ]:
len(iID_list) #total number of patients in initial query

In [ ]:
start_levo = 6.0

In [ ]:
time_thres = 24.0

In [ ]:
buffer_time = 3.0

In [ ]:
#initialize variables
ml = pd.DataFrame(columns = ['icu_stayID','first_lact_time','final_lact_time','initial_lact','final_lact','levo_time','vasopressin','vaso_time','vaso_dose','hosp_exp'])
remove_id = [] #list of icustays for patients who don't have the correct timing of lactate measurements
short_lact = [] #list of icustays for patients who have only 1 lactate value
lact_traject_time = {} #dictionary of lactate trajectories
lact_traject_val = {} #dictionary of lactate values
#one_val = pd.DataFrame(columns = ['icu_stayID', 'lact_val']) #patients with only one lactate value
#wrong_time = pd.DataFrame(columns = ['icu_stayID', 'last_time', 'last_lact','vasopressin', 'vaso_time', 'vaso_dose','dod']) #patients who do not meet the timing requirements

In [ ]:
# Check inclusion criteria and populate new dataframe
for i in range(len(iID_list)): #loop through each paitent
    ids = iID_list[i] #select a icustay ID
    n = iID_count[i] #find the number of entries for that ID
    time = data.time_since_admit[iID_ind[i]:iID_ind[i]+n] #get the lactate measurement times
    lac = data.valuenum[iID_ind[i]:iID_ind[i]+n] #get the lactate measurement value
    dod = data.time_of_death[iID_ind[i]] #pull the dod
    hosp_exp = data.hospital_expire_flag[iID_ind[i]] #pull the expiry flag
    #find if the patient is in the metavision or carevue results for Levophed
    #first check metavision
    if not mvLevo.loc[mvLevo.icustay_id == ids].time_since_admit.empty and len(lac) >= 2 and data.age_on_admit[iID_ind[i]] >= 18.0: #test if array is empty
        
        mv_time = min(mvLevo.loc[mvLevo.icustay_id == ids].time_since_admit)
        start_window = time.loc[time - mv_time <  buffer_time]
        start_window = start_window.loc[time - mv_time > -0.5]
        if start_window.empty or mv_time > start_levo:
            remove_id.append(ids) #no lactate monitoring when patient receives Levophed
        else:
            t0_ind = start_window.idxmin()
            t0 = time[t0_ind]
            window = time.loc[(time - mv_time) >= (time_thres - buffer_time)]
            window = window.loc[(time - mv_time) <= (time_thres + buffer_time)]
            if window.empty:
                remove_id.append(ids)
            else:
                val_end = abs(window - time_thres)
                tf = val_end.idxmin()
                lact_traject_time[ids] = time
                lact_traject_val[ids] = lac
                ml = ml.append({'icu_stayID': ids, 'first_lact_time': t0, 'final_lact_time': time[tf], 'initial_lact':lac[t0_ind], 'final_lact':lac[tf], 'levo_time': mv_time, 'vasopressin':0, 'vaso_time':0, 'vaso_dose':0, 'hosp_exp':hosp_exp}, ignore_index = True)
    
    elif not cvLevo.loc[cvLevo.icustay_id == ids].time_since_admit.empty and len(lac) >= 2 and data.age_on_admit[iID_ind[i]] >= 18.0: #test if array is empty
        
        cv_time = min(cvLevo.loc[cvLevo.icustay_id == ids].time_since_admit)
        start_window = time.loc[time - cv_time <  buffer_time]
        start_window = start_window.loc[time - cv_time > -0.5]
        if start_window.empty or cv_time > start_levo:
            remove_id.append(ids) #no lactate monitoring when patient receives Levophed
        else:
            t0_ind = start_window.idxmin()
            t0 = time[t0_ind]
            window = time.loc[(time - cv_time) >= (time_thres - buffer_time)]
            window = window.loc[(time - cv_time) <= (time_thres + buffer_time)]
            if window.empty:
                remove_id.append(ids)
            else:
                val_end = abs(window - time_thres)
                tf = val_end.idxmin()
                lact_traject_time[ids] = time
                lact_traject_val[ids] = lac
                ml = ml.append({'icu_stayID': ids, 'first_lact_time': t0, 'final_lact_time': time[tf], 'initial_lact':lac[t0_ind], 'final_lact':lac[tf], 'levo_time': cv_time, 'vasopressin':0, 'vaso_time':0, 'vaso_dose':0, 'hosp_exp':hosp_exp}, ignore_index = True)
                
#end loop through icustay IDs

In [ ]:
plt.subplot(1,3,1)
plt.hist(ml.levo_time)
plt.title('Levophed Started')
plt.xlabel('Time since admit (hrs)')

plt.subplot(1,3,2)
plt.hist(ml.first_lact_time - ml.levo_time)
plt.title('Relative lactate start time')
plt.xlabel('Time since admit (hrs)')

plt.subplot(1,3,3)
plt.hist(ml.first_lact_time)
plt.title('Lactate start time')
plt.xlabel('Time since admit (hrs)')

print len(ml)
print min(ml.levo_time)
print max(ml.levo_time)
print max(ml.first_lact_time)

In [ ]:
plt.subplot(1,2,1)
plt.hist(ml.final_lact_time - ml.levo_time)
plt.title('Final lactate measurement relative to Levophed start')
plt.xlabel('Time since admit (hrs)')

plt.subplot(1,2,2)
plt.hist(ml.final_lact_time)
plt.title('Final lactate measurement')
plt.xlabel('Time since admit (hrs)')

print min(ml.final_lact_time - ml.levo_time)
print max(ml.final_lact_time - ml.levo_time)

In [ ]:
plt.plot(ml.levo_time,ml.first_lact_time,'o')
plt.plot(np.arange(-1,9),np.arange(-1,9),'k')
plt.plot(np.arange(-1,9),np.arange(-1,9) - 0.5, 'k--')
plt.plot(np.arange(-1,9),np.arange(-1,9) + 3.0, 'k--')
plt.xlabel('Time levophed is first administered')
plt.ylabel('Time of recorded lactate measurement')

In [ ]:
for k in ml.icu_stayID:
    plt.plot(lact_traject_time[k],lact_traject_val[k],'o-')
    plt.xlim(-6,30)
    plt.ylim(0,20)

# Query for vasopressin information
The below query pulls information from MIMIC concerning vasopressin administration

In [ ]:
# query the metavision table for the medication information
con = psycopg2.connect(dbname=dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path TO ' + schema_name)

query = \
"""
SELECT icu.icustay_id, ie.itemid, d.label, ie.starttime, ie.rate, ie.rateuom, ie.amount, ie.amountuom
     , EXTRACT (EPOCH FROM (ie.starttime - icu.intime)/60.0/60.0) AS time_since_admit -- time in hours
FROM icustays icu
LEFT JOIN inputevents_mv ie
ON icu.icustay_id = ie.icustay_id
LEFT JOIN d_items d
ON ie.itemid = d.itemid
WHERE ie.itemid IN (
    222315 -- Vasopressin
    ) 
AND ie.rate = ie.rate
ORDER BY icu.icustay_id, ie.starttime
;"""

vaso_data = pd.read_sql_query(query,con)

In [ ]:
start_vaso = 3.0 #starting time for vasopressin window
end_vaso = 18.0 #ending time for vasopressin window

In [ ]:
# go through all of the icustay_ids and determine if vasopressin was administered between hour 3 and 18
# THIS ONE IS FOR METAVISION
early_sum = 0 #store the number of patients who received vasopressin before hour 3
late_sum = 0 #store the number of patients who received vasopressin after hour 18
for i in ml.icu_stayID: #loop through each icustay_id
    meds = vaso_data.loc[vaso_data.icustay_id == i] #get a list of the medications the patient receives
    if not meds.empty:
        if meds['time_since_admit'].iloc[0] < start_vaso: #float(ml.loc[ml.icu_stayID == i].first_lact_time):
            ml = ml[ml.icu_stayID != i] #drop patients who receive vasopressin prior to lactate monitoring
            early_sum = early_sum + 1
        elif meds['time_since_admit'].iloc[0] > end_vaso: #float(ml.loc[ml.icu_stayID == i].first_lact_time) + 24.0:
            ml = ml[ml.icu_stayID != i] #drop patients who start receiving vasopressin 24 hours after first lactate
            late_sum = late_sum + 1
        else:
            ml.loc[ml.icu_stayID == i, 'vaso_time'] = meds['time_since_admit'].iloc[0]
            if meds['rateuom'].iloc[0] == 'units/hour':
                ml.loc[ml.icu_stayID == i, 'vaso_dose'] =meds['rate'].iloc[0]/60
            else:
                ml.loc[ml.icu_stayID == i, 'vaso_dose'] =meds['rate'].iloc[0]
            ml.loc[ml.icu_stayID == i,'vasopressin'] = 1 #populate the data table
              

In [ ]:
print early_sum
print late_sum

In [ ]:
# this is the carevue version of the query
con = psycopg2.connect(dbname=dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path TO ' + schema_name)

query = \
"""
SELECT icu.icustay_id, ie.itemid, d.label, ie.charttime, ie.rate, ie.rateuom
     , EXTRACT (EPOCH FROM (ie.charttime - icu.intime)/60.0/60.0) AS time_since_admit -- time in hours
FROM icustays icu
LEFT JOIN inputevents_cv ie
ON icu.icustay_id = ie.icustay_id
LEFT JOIN d_items d
ON ie.itemid = d.itemid
WHERE ie.itemid IN (
    1136, 1327, 2334, 42273, 1222, 2248, 2561, 2765, 42802, 2445, 7341, 30051, 222315 -- Vasopressin
    ) 
AND ie.rate = ie.rate
ORDER BY icu.icustay_id, ie.charttime
;"""

vaso_data = pd.read_sql_query(query,con)
np.unique(vaso_data.rateuom)
#cvdata.loc[cvdata.itemid != 30051] #note that only 30051 returns vasopressin results

In [ ]:
# go through all of the icustay_ids and determine if vasopressin is administered between hour 3 and 18
# THIS ONE IS FOR CAREVUE
vaso_data = vaso_data[vaso_data.rate != 0] #first remove entries where no vasopressin is administered
for i in ml.icu_stayID: #loop through each icustay_id
    meds = vaso_data.loc[vaso_data.icustay_id == i] #get a list of the medications the patient receives
    if not meds.empty:
        if meds['time_since_admit'].iloc[0] < start_vaso: #float(ml.loc[ml.icu_stayID == i].first_lact_time):
            ml = ml[ml.icu_stayID != i] #drop patients who receive vasopressin prior to lactate monitoring
            early_sum = early_sum + 1
        elif meds['time_since_admit'].iloc[0] > end_vaso: #float(ml.loc[ml.icu_stayID == i].first_lact_time) + 24.0:
            ml = ml[ml.icu_stayID != i] #drop patients who start receiving vasopressin 24 hours after first lactate
            late_sum = late_sum + 1
        else:
            ml.loc[ml.icu_stayID == i, 'vaso_time'] = meds['time_since_admit'].iloc[0]
            if meds['rateuom'].iloc[0] == 'Uhr':
                ml.loc[ml.icu_stayID == i, 'vaso_dose'] = meds['rate'].iloc[0]/60
            else:
                ml.loc[ml.icu_stayID == i, 'vaso_dose'] = meds['rate'].iloc[0]
            ml.loc[ml.icu_stayID == i,'vasopressin'] = 1 #populate the data table
            

In [ ]:
print early_sum
print late_sum

In [ ]:
print len(ml)
print sum(ml.vasopressin)

plt.hist(ml.loc[ml.vaso_time != 0.0].vaso_time)
plt.xlabel('Vasopressin time since admit (hrs)')

In [ ]:
plt.subplot(1,3,1)
plt.plot(ml.first_lact_time,ml.levo_time,'o')
plt.xlabel('First lactate')
plt.ylabel('Levo time')

plt.subplot(1,3,2)
plt.plot(ml.loc[ml.vaso_time != 0.0].first_lact_time,ml.loc[ml.vaso_time != 0.0].vaso_time,'o')
plt.xlabel('First lactate')
plt.ylabel('Vaso time')

plt.subplot(1,3,3)
plt.plot(ml.loc[ml.vaso_time != 0.0].levo_time,ml.loc[ml.vaso_time != 0.0].vaso_time,'o')
plt.plot(np.arange(-1,20),np.arange(-1,20),'k')
plt.xlim(-1,6)
plt.xlabel('Levo time')
plt.ylabel('Vaso time')

In [ ]:
ml.to_csv('matched_total_LevoTiming.csv')

# Covariate Queries
The below queries gather the covariate information for the study. There is an intemediate step to build 'matched_cohort SICU' which is performed in a separate script.

In [ ]:
con = psycopg2.connect(dbname=dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path TO ' + schema_name)

query = \
"""
SELECT l.icustay_id, i.first_careunit, aps.apsiii, l.first_lact_time, l.first_lact, l.last_lact, l.levo_time
    , a.admission_type, a.admission_location
    , l.vaso_time, p.gender, a.ethnicity, a.hospital_expire_flag
    , EXTRACT (EPOCH FROM (i.intime - p.dob)/60.0/60.0/24.0/365.242) AS age_on_admit -- patient age in years
    , e.renal_failure, e.liver_disease, e.congestive_heart_failure, lb.creatinine_mean, vt.heartrate_mean
    , vt.meanbp_mean, vt.spo2_mean, vn.mechvent, ang.angus, ang.explicit_sepsis, l.vaso, det.los_hospital --, bg.fio2
FROM matched_cohort_LevoTime l
INNER JOIN icustays i
ON l.icustay_id = i.icustay_id
LEFT JOIN apsiii_first3hours aps
ON l.icustay_id = aps.icustay_id
LEFT JOIN admissions a
ON i.hadm_id = a.hadm_id
LEFT JOIN icustay_detail det
ON i.icustay_id = det.icustay_id
LEFT JOIN patients p
ON i.subject_id = p.subject_id
LEFT JOIN elixhauser_ahrq e
ON i.hadm_id = e.hadm_id
LEFT JOIN angus_sepsis ang
ON i.hadm_id = ang.hadm_id
LEFT JOIN labsfirst3hours lb
ON l.icustay_id = lb.icustay_id
LEFT JOIN vitalsfirst3hours vt
ON l.icustay_id = vt.icustay_id
LEFT JOIN ventfirst3hours vn
ON l.icustay_id = vn.icustay_id;"""
#WHERE l.clear_int != 2;"""
#LEFT JOIN bloodgasfirstday bg
#ON l.icustay_id = bg.icustay_id;"""

data = pd.read_sql_query(query,con)
data #expect 268 rows

In [ ]:
# Initialize additional data columns
data['NS'] = 0
data['fio2'] = 0

In [ ]:
# metavision query for normal saline
query = \
"""
SELECT c.icustay_id, ie.itemid, d.label
    , ie.starttime, EXTRACT (EPOCH FROM (ie.starttime - icu.intime)/60.0/60.0) AS time_since_admit -- time in hours
    , ie.amount, ie.amountuom
FROM matched_cohort_LevoTime c 
LEFT JOIN icustays icu
ON c.icustay_id = icu.icustay_id
LEFT JOIN inputevents_mv ie
ON c.icustay_id = ie.icustay_id
LEFT JOIN d_items d
ON ie.itemid = d.itemid
WHERE ie.itemid IN (
    225158 )-- Metavision
ORDER BY c.icustay_id, ie.starttime;"""

mvfluidNS = pd.read_sql_query(query,con)
mvfluidNS.loc[mvfluidNS.amountuom != 'ml']

In [ ]:
# go through all of the icustay_ids and determine if the patient is receiving normal saline
# THIS ONE IS FOR metavision
for i in data.icustay_id: #loop through each icustay_id
    times = mvfluidNS.loc[mvfluidNS.icustay_id == i] #get the subset of data for the particular patient
    if not times.empty:
        data.loc[data.icustay_id == i,'NS'] = sum(times.loc[times.time_since_admit < 3.0].amount)

In [ ]:
#carevue query for normal saline
query = \
"""
SELECT c.icustay_id, ie.itemid, d.label
    , ie.charttime, EXTRACT (EPOCH FROM (ie.charttime - icu.intime)/60.0/60.0) AS time_since_admit -- time in hours
    , ie.amount, ie.amountuom
FROM matched_cohort_LevoTime c 
LEFT JOIN icustays icu
ON c.icustay_id = icu.icustay_id
LEFT JOIN inputevents_cv ie
ON c.icustay_id = ie.icustay_id
LEFT JOIN d_items d
ON ie.itemid = d.itemid
WHERE ie.itemid IN (
    44633, 5321, 41237, 41371, 41428, 40850, 30352, 30018, 4647, 5333, 6190, 4970, 
    5199, 41913, 40865, 44983, 30190, 44741, 41467, 45989, 41490, 44491, 41695, 
    42548, 42844, 45137, 44440 -- Carevue
    ) 
AND ie.amount = ie.amount
AND ie.amount <> 0.0
ORDER BY c.icustay_id, ie.charttime
;"""

cvfluidNS = pd.read_sql_query(query,con)
cvfluidNS.loc[cvfluidNS.amountuom != 'ml']

In [ ]:
# go through all of the icustay_ids and determine if the patient is receiving normal saline
# THIS ONE IS FOR CAREVUE
for i in data.icustay_id: #loop through each icustay_id
    times = cvfluidNS.loc[cvfluidNS.icustay_id == i] #get the subset of data for the particular patient
    if not times.empty:
        data.loc[data.icustay_id == i,'NS'] = sum(times.loc[times.time_since_admit < 3.0].amount)

In [ ]:
# query for fi02
query = \
"""
SELECT m.icustay_id, bg.charttime, bg.fio2,
    EXTRACT (EPOCH FROM (bg.charttime - icu.intime)/60.0/60.0) AS time_since_admit -- time in hours
FROM matched_cohort_LevoTime m
INNER JOIN bloodgasfirstday bg
ON m.icustay_id = bg.icustay_id
INNER JOIN icustays icu
ON m.icustay_id = icu.icustay_id
WHERE bg.fio2 <> 0.0
ORDER BY m.icustay_id, bg.charttime
;"""
fi02data = pd.read_sql_query(query,con)


In [ ]:
# go through all of the icustay_ids and determine if the patient is receiving lactate ringers
# THIS ONE IS FOR CAREVUE
for i in data.icustay_id: #loop through each icustay_id
    times = fi02data.loc[fi02data.icustay_id == i] #get the subset of data for the particular patient
    if not times.loc[times.time_since_admit < 3.0].empty:
        data.loc[data.icustay_id == i,'fio2'] = times.loc[times.time_since_admit < 3.0].fio2.mean()

In [ ]:
data.to_csv('basic_set_LevoTime_APS3.csv')